In [1]:
# local imports
from src.transfer import replace_head, METRICS
from src.data import player_data
from src.util import display_progress

# computation / deep learning imports
from tensorflow import keras
import numpy as np

# os / filesystem imports
import pickle
import os

# visualization imports
from sklearn.metrics import confusion_matrix
import pandas as pd

# Model with Pre-Trained Base

In [2]:
# load model
model = keras.models.load_model('models/SSBML-Base-Model')

# replace head
model = replace_head(model)

In [3]:
model.summary()

Model: "SSBML-Transfer-Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SSBML-Base-Model (Sequential (None, 512)               6537842   
_________________________________________________________________
Binary-Classifier (Sequentia (None, 1)                 83329     
Total params: 6,621,171
Trainable params: 82,817
Non-trainable params: 6,538,354
_________________________________________________________________


# The Players

In [4]:
!ls data/player

Blynde	CuckDaddy  gh0st  ixwonkr  Lie0x  TCBL


# The Player Data

Player Data (clips played by our chosen player) can be found in data/player/\<player name\>

Anonymous Data (clips not played by our chosen player) is taken from the large dataset data/character

In [5]:
# name of the player we want to train/test on
player_name = 'TCBL'

In [6]:
# set all filepath related variables

player_dir = os.path.join('data/player', player_name)
player_train_dir = os.path.join(player_dir, 'train')
player_test_dir = os.path.join(player_dir, 'test')
player_train_sample_size = len(os.listdir(player_train_dir))
player_test_sample_size = len(os.listdir(player_test_dir))

anonymous_dir = 'data/character'
anonymous_train_dir = os.path.join(anonymous_dir, 'train')
anonymous_test_dir = os.path.join(anonymous_dir, 'test')

print(f'Training Data Located at: \n\t- {player_train_dir} \n\t- {anonymous_train_dir} \n')
print(f'Testing Data Located at: \n\t- {player_test_dir} \n\t- {anonymous_test_dir} \n')
print(f'Player Training Data Sample size: \n\t- {player_train_sample_size} \n')
print(f'Player Testing Data Sample size: \n\t- {player_test_sample_size} \n')

Training Data Located at: 
	- data/player/TCBL/train 
	- data/character/train 

Testing Data Located at: 
	- data/player/TCBL/test 
	- data/character/test 

Player Training Data Sample size: 
	- 1822 

Player Testing Data Sample size: 
	- 222 



# Training

Train the model on a mix of the chosen player's clips,
and random anonymous clips from the Melee Public SLP Dataset

## Data Generation

In [7]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Anonymous Clips / Chosen Player's Clips
class_balance_ratio = 5

# Affects batch_size and steps_per_epoch
# Example: ratio of 2 would effectively 
#          double batch size and 
#          cut steps_per_epoch in half
tuning_ratio = 1

# =====================

# Calculate number of steps per epoch for train/test loops.
# One Epoch should iterate through our player's clips once, mixing them
# with random anonymous clips at our given class balance ratio 
train_steps = (
    (player_train_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)
    
test_steps = (
    (player_test_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)

# training data
training_data = player_data(
    player_train_dir,
    anonymous_train_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

# testing data
testing_data = player_data(
    player_test_dir,
    anonymous_test_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

## Training Loop

In [8]:
model.fit(
    training_data,
    epochs = 10,
    steps_per_epoch = train_steps,
    verbose = 1,
);

Epoch 1/10
683/683 [==============================] - 174s 252ms/step - loss: 0.0417 - accuracy: 0.8586 - precision: 0.5773 - recall: 0.3647 - specificity: 0.9440
Epoch 2/10
683/683 [==============================] - 172s 252ms/step - loss: 0.0211 - accuracy: 0.9157 - precision: 0.8720 - recall: 0.5859 - specificity: 0.9826
Epoch 3/10
683/683 [==============================] - 175s 257ms/step - loss: 0.0178 - accuracy: 0.9272 - precision: 0.8767 - recall: 0.6412 - specificity: 0.9825
Epoch 4/10
683/683 [==============================] - 174s 255ms/step - loss: 0.0168 - accuracy: 0.9313 - precision: 0.9120 - recall: 0.6680 - specificity: 0.9865
Epoch 5/10
683/683 [==============================] - 174s 256ms/step - loss: 0.0155 - accuracy: 0.9351 - precision: 0.9005 - recall: 0.6974 - specificity: 0.9841
Epoch 6/10
683/683 [==============================] - 172s 252ms/step - loss: 0.0144 - accuracy: 0.9428 - precision: 0.9124 - recall: 0.7306 - specificity: 0.9858
Epoch 7/10
683/683 [==

In [9]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Anonymous Clips / Chosen Player's Clips
class_balance_ratio = 3

# Affects batch_size and steps_per_epoch
# Example: ratio of 2 would effectively 
#          double batch size and 
#          cut steps_per_epoch in half
tuning_ratio = 1

# =====================

# Calculate number of steps per epoch for train/test loops.
# One Epoch should iterate through our player's clips once, mixing them
# with random anonymous clips at our given class balance ratio 
train_steps = (
    (player_train_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)
    
test_steps = (
    (player_test_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)

# training data
training_data = player_data(
    player_train_dir,
    anonymous_train_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

# testing data
testing_data = player_data(
    player_test_dir,
    anonymous_test_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

In [10]:
model.fit(
    training_data,
    epochs = 5,
    steps_per_epoch = train_steps,
    verbose = 1,
);

Epoch 1/5
455/455 [==============================] - 117s 257ms/step - loss: 0.0160 - accuracy: 0.9348 - precision: 0.9318 - recall: 0.8045 - specificity: 0.9797
Epoch 2/5
455/455 [==============================] - 116s 256ms/step - loss: 0.0160 - accuracy: 0.9378 - precision: 0.9318 - recall: 0.8124 - specificity: 0.9800
Epoch 3/5
455/455 [==============================] - 115s 253ms/step - loss: 0.0161 - accuracy: 0.9364 - precision: 0.9324 - recall: 0.8058 - specificity: 0.9804
Epoch 4/5
455/455 [==============================] - 116s 255ms/step - loss: 0.0155 - accuracy: 0.9375 - precision: 0.9261 - recall: 0.8092 - specificity: 0.9791
Epoch 5/5
455/455 [==============================] - 117s 256ms/step - loss: 0.0150 - accuracy: 0.9385 - precision: 0.9352 - recall: 0.8098 - specificity: 0.9813


In [11]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Anonymous Clips / Chosen Player's Clips
class_balance_ratio = 2

# Affects batch_size and steps_per_epoch
# Example: ratio of 2 would effectively 
#          double batch size and 
#          cut steps_per_epoch in half
tuning_ratio = 2

# =====================

# Calculate number of steps per epoch for train/test loops.
# One Epoch should iterate through our player's clips once, mixing them
# with random anonymous clips at our given class balance ratio 
train_steps = (
    (player_train_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)
    
test_steps = (
    (player_test_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)

# training data
training_data = player_data(
    player_train_dir,
    anonymous_train_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

# testing data
testing_data = player_data(
    player_test_dir,
    anonymous_test_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

In [12]:
model.fit(
    training_data,
    epochs = 2,
    steps_per_epoch = train_steps,
    verbose = 1,
);

Epoch 1/2
170/170 [==============================] - 67s 395ms/step - loss: 0.0147 - accuracy: 0.9419 - precision: 0.9550 - recall: 0.8686 - specificity: 0.9792
Epoch 2/2
170/170 [==============================] - 67s 396ms/step - loss: 0.0142 - accuracy: 0.9500 - precision: 0.9568 - recall: 0.8905 - specificity: 0.9799


In [13]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Anonymous Clips / Chosen Player's Clips
class_balance_ratio = 1

# Affects batch_size and steps_per_epoch
# Example: ratio of 2 would effectively 
#          double batch size and 
#          cut steps_per_epoch in half
tuning_ratio = 4

# =====================

# Calculate number of steps per epoch for train/test loops.
# One Epoch should iterate through our player's clips once, mixing them
# with random anonymous clips at our given class balance ratio 
train_steps = (
    (player_train_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)
    
test_steps = (
    (player_test_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)

# training data
training_data = player_data(
    player_train_dir,
    anonymous_train_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

# testing data
testing_data = player_data(
    player_test_dir,
    anonymous_test_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

In [14]:
model.fit(
    training_data,
    epochs = 1,
    steps_per_epoch = train_steps,
    verbose = 1,
);

56/56 [==============================] - 39s 689ms/step - loss: 0.0144 - accuracy: 0.9459 - precision: 0.9757 - recall: 0.9149 - specificity: 0.9770


# Testing

Test the model on one pass of the given player's clips, 
at the given class balance ratio

In [15]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Ratio of Anonymous clips : Chosen Player's clips
class_balance_ratio = 1

# =====================

data_test_one_round = player_data(
    player_test_dir,
    anonymous_test_dir,
    batch_size = DEFAULT_BATCH_SIZE,
    repeat = False,
    ratio = class_balance_ratio,
)

score = model.evaluate(data_test_one_round, verbose=1)
print('\nTest score:', round(score[0], 3))
print(f'- accuracy: {round(score[1]*100)}%')
print(f'- precision: {round(score[2]*100)}%')
print(f'- recall: {round(score[3]*100)}%')
print(f'- specificity: {round(score[4]*100)}%')

29/29 [==============================] - 7s 210ms/step - loss: 0.0141 - accuracy: 0.9591 - precision: 0.9668 - recall: 0.9444 - specificity: 0.9718

Test score: 0.014
- accuracy: 96%
- precision: 97%
- recall: 94%
- specificity: 97%


## Confusion Matrix

In [16]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Ratio of Anonymous clips : Chosen Player's clips
class_balance_ratio = 1

# =====================

# define data generation
data_conf = player_data(
    player_test_dir,
    anonymous_test_dir,
    batch_size = DEFAULT_BATCH_SIZE,
    ratio = class_balance_ratio,
)

# predict over test data
batch_preds = []
batch_labels = []
i = 0
N = player_test_sample_size
for xi, yi in data_conf:
    batch_preds.append(model.predict(xi).round().astype(int))
    batch_labels.append(yi.astype(int))
    
    # progess bar
    i = int(i + np.sum(yi))
    display_progress(i, N)
display_progress(N, N)
print('\n')

pred = np.concatenate(batch_preds)
labels = np.concatenate(batch_labels)

# create confusion matrix
# reverse so true positive is top left, true negative is bottom right
conf_matrix = confusion_matrix(labels, pred, normalize='all')[::-1, ::-1] 
conf_matrix = np.around(conf_matrix, 3)
conf_df = pd.DataFrame(
    conf_matrix, 
    index = ['present', 'not present'], 
    columns = ['detected', 'not detected']
)

conf_df

[####################] 222 of 222 - 100.0% 



,detected,not detected
present,0.490,0.031
not present,0.026,0.452
